## About this test
This test shall take as input a BluePyOpt optimized output file, containing a `hall_of_fame.json` file specifying a collection of parameter sets. The validation test would then evaluate the model for all (or specified) parameter sets against various eFEL features. The results are registered on the HBP Validation Framework app. If an instance of the Model Catalog and Validation Framework are not found in the current Collab, then these are created. Additionally, a test report is generated and this can be viewed within the Jupyter notebook, or downloaded.

### Installing the required packages
(no user intervention required)

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version

req_packages = {    
                    "hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"},
                    "sciunit"                  : {"min_version": "0.19",   "install_version": "https://github.com/scidash/sciunit"},
                    "bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"},
                    "basalunit"                : {"min_version": "1.0.1",  "install_version": "1.0.1"},
                    "hbp_validation_framework" : {"min_version": "0.4",    "install_version": "0.4.1"},
                    "fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"},
                    "PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"}
               }

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg_name, pkg_vinfo in req_packages.items():
        print("Checking for package: {}".format(pkg_name))
        try:
            pkg_resources.get_distribution(pkg_name)
            if pkg_vinfo["min_version"]:
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                if current_version < parse_version(pkg_vinfo["min_version"]):
                    print("\t{}: current version = {}".format(pkg_name, current_version))
                    print("\tA newer version of {} will be installed.".format(pkg_name))
                    raise
        except:
            if "github.com" in pkg_vinfo["install_version"]:
                os.system("pip install --quiet git+{}".format(pkg_vinfo["install_version"]))                
            else:
                os.system("pip install --quiet {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                
            if pkg_name == "hbp_service_client":
                from IPython.display import HTML
                display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))
                
install_req_packages()                 

import json
import sciunit
import bluepyopt.ephys as ephys
from basalunit.utils import CellModel
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

### Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME)
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_collab_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_collab_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

### Model Selection
The user is given an option to choose from existing models, or to specify a new model. The former are currently sourced from https://collab.humanbrainproject.eu/#/collab/8123/nav/61645 (see: `/8123/usecases/models/basal_ganglia/`). Only models saved at this location are shown as existing models.

If specifying a new model, the path to the model within Collab storage needs to be specified.<br />
E.g. `/8123/usecases/models/msn_d1_all-6-20170524.tar.gz`

In [ ]:
clients = get_hbp_service_client()
entries = clients.storage.list("/8123/usecases/models/basal_ganglia/")
entries.append("Use own model (Collab path to be specified)")
print("Example models are listed below:")
for i, each in enumerate(entries,start=1):
    print ("\t{}. {}".format(i,each))
print("Enter the # of required model: ")
choice = input()
if choice < len(entries):    
    file_path = "/tmp/" + entries[choice-1]
    clients.storage.download_file("/8123/usecases/models/basal_ganglia/" + entries[choice-1], file_path)
else:
    print("Please enter the Collab storage path to the BluePyOpt output zip/tar file")
    print("e.g. '/8123/usecases/models/msn_d1_all-6-20170524.tar.gz'")
    collab_path = raw_input()
    file_path = "/tmp/" + os.path.basename(collab_path)
    clients.storage.download_file(collab_path, file_path)    

model_name = os.path.basename(file_path).split(".")[0]
print("Model Name = {}".format(model_name))

### Gather Additional Info
Currently the test is applicable for three cell types: _D1-type MSN_, _D2-type MSN_ and _FS Interneurons_ <br />
The usecase tries to identify the cell type from the model name. If this isn't possible, the user is prompted to specify the cell type.

Experimental recordings might specify a value for their junctional potential. For the observation data used here, the same was recorded as 9.5 mV and is therefore used as the default value. The user is prompted to change this if needed.

In [ ]:
# will try to determine cell type from file name; if not then ask user
# current options: msn_d1, msn_d2, fs
if "msn_d1" in model_name:
    cell_type = "msn_d1" 
elif "msn_d2" in model_name:
    cell_type = "msn_d2"
elif "-fs-" in model_name:
    cell_type = "fs"    
else:
    print("\nPlease enter the cell_type: ")
    options = ["msn_d1", "msn_d2", "fs"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = input()
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))
    
print("\nEnter a value for junction potential of experimental recordings (in mV):")
print("Note: leave blank to use default value (i.e. 9.5); enter numeric value, no units")
jp_value = raw_input()
if jp_value == "":
    junction_potential = 9.5 # mV
else:
    junction_potential = float(jp_value)
print("Junction Potential = {} mV".format(junction_potential))

### Specifying model from ModelCatalog; Update info if new model entry
For a model to use the validation framework, it first has to be registered on the _Model Catalog_ app (see Collab's Navigation panel; select `Model Catalog`) <br />

If the model has already been registered on the model catalog, then you simply need to specify the `model_uuid`. <br /> E.g. c7748201-5551-4d56-bc31-f7feec5e5798 <br />

If this is to be registered as a new model, then you are required to update the cell below to alter the model metadata as required. See [here](http://hbp-validation-client.readthedocs.io/en/master/#hbp_validation_framework.ModelCatalog.register_model) for detailed info on the various parameters. Once a new model is registered on the model catalog, it will show you the corresponding `model_uuid`. This can be used for future tests to link the results with the same model entry.

In [ ]:
print("\nDo you wish to use an existing `model_uuid` (this will register results against an existing model entry)?")
print("Note: `model_uuid` is obtained from the HBP Validation Framework app.")
print("If not specifying `model_uuid`, then a new model entry would be registered.")
print("Note: If registering a new model, please verify the default model metadata in this Jupyter notebook!")
print("Enter: y/n")
choice = raw_input().lower()
valid_choices = {"yes": True, "y": True, "no": False, "n": False}
if valid_choices[choice]:
    print("\nPlease enter a valid `model_uuid`: ")
    model_uuid = raw_input()
    # Verify the user specified `model_uuid` is valid
    try:
        modelCatalog.get_model(model_id=model_uuid)
    except:
        raise ValueError("Specified model_uuid = {} is invalid!".format(model_uuid))
else:
    # Create a new model entry in the ModelCatalog; metadata to be used for registering model
    model_uuid = modelCatalog.register_model(
                                app_id=str(MCapp_navID),
                                name=model_name,
                                # alias="", # optional; ensure uniqueness                                
                                author="Alexander Kozlov",
                                organization="KTH-UNIC",
                                private=False,
                                cell_type="Other",
                                model_type="Single Cell",
                                brain_region="Basal Ganglia",
                                species="Mouse (Mus musculus)",
                                description="This model is being used to demonstrate use of the Validation Service")
    model_uuid = str(model_uuid)
    print("\nModel registered on the HBP Validation Framework's Model Catalog!")
    print("Note: you can specify the below `model_uuid` for future runs of this test.")
print("\nmodel_uuid = {}".format(model_uuid))
print("Link to ModelCatalog app:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=model.{}".format(str(collab_id),str(MCapp_navID),model_uuid))

### Gathering Hall of Fame (HoF) Parameter Sets
This usecase expects to find a file named `hall_of_fame.json` inside the model's base directory. This file should contain the various hall of fame parameter sets that are to be tested.

The model morphology file (.swc) that was found within the model directory, and used to construct the model, is indicated.

In [ ]:
result_uuids = []

# change 'model_name' below to specify your own; else file base name is used
cell_model = CellModel(model_path=file_path, cell_type=cell_type, model_name=model_name)
cell_model.model_uuid = model_uuid

hall_of_fame = json.load(open(os.path.join(cell_model.base_path, 'hall_of_fame.json')))

### Specifying the parameter sets of interest; Running model validation for these sets
The user is asked to specify the _Hall of Fame_ parameter sets that they are interested in validating. The valid range of values are indicated. The user can provide input as follows: <br />
 * All parameters, specify: `all` <br />
 * Single parameter set, specify number, e.g.: `0` <br />
 * Multiple parameter sets, specify numbers as a list, e.g.: `[0,4,8,9]` <br />
 
The usecase will then run validations for these parameter sets. Do note that it takes roughly 20 minutes for each parameter set. At the end of the test, the user is provided with a textual summary of the _score_ and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
# cycling through hall of fame
print("\nTotal available 'Hall of Fame' parameter sets = {}".format(len(hall_of_fame)))
print("Note: validation of each parameter set takes around 20 mins; for testing specify a single parameter set.")
print("Enter the parameter set of interest: 0 - {}".format(len(hall_of_fame)-1))
print("Example inputs: 0, 4, [0,4,8,9], all")
param_entry = raw_input().lower()
if param_entry == "all":
    param_sets = range(len(hall_of_fame))
else:    
    if isinstance(eval(param_entry), list):
        param_sets = eval(param_entry)
    elif isinstance(eval(param_entry), int):
        param_sets = [eval(param_entry)]
    else:
        raise ValueError("Invalid entry for parameter set!")

for hof_index in param_sets:
    cell_model.version = cell_model.model_name + "_hof_" + str(hof_index)
    cell_model.params = hall_of_fame[hof_index]
        
    result_id = utils.run_test(hbp_username=HBP_USERNAME, environment="production", model=cell_model, test_alias="basalg_" + cell_type, test_version="1.0", storage_collab_id=collab_id, register_result=True, client_obj=testLibrary, cell_type=cell_type, junction_potential=junction_potential, use_cache=True)
    result_uuids.append(result_id)    

print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### Generate Report
The validation framework can generate a PDF report for all model variants (HoF parameter sets employed) that successfully completed the test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        if not HBP_USERNAME:
            print("Enter your HBP username: ")
            HBP_USERNAME = raw_input()            
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="production", result_list=result_uuids, only_combined=True)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))